In [1]:
# Import necessary libraries
import random, time, os
import mysql.connector
from dotenv import load_dotenv

load_dotenv()

True

# Problem 2 implementation: Simplified version of Yannakakis' algorithm

In [2]:
def construct_hashmap_problem2(arr, index):
    """
    Docstring for construct_hashmap_problem2
    
    :param arr: a 2D array
    :param index: the index to be used as key. Only 0 or 1.
    :return: a hashmap where the keys are the elements at the given index.
    """
    h = {}
    for i in range(len(arr)):
        cur_element = arr[i]
        key = cur_element[index]
        val = cur_element[1 - index]
        if key not in h:
            h[key] = [val]
        else:
            h[key].append(val)
    return h

def remove_dangling_tuple(db):
    """
    Docstring for remove_dangling_tuple
    
    :param db: a 3D array. Each element is a 2D array representing a table.
    """
    h_bottom_up = {i: construct_hashmap_problem2(db[i], 1) for i in range(len(db))}
    h_top_down = {i: construct_hashmap_problem2(db[i], 0) for i in range(len(db))}
    
    # Bottom up pass
    for i in range(len(db)-2, -1, -1):
        for j in range(len(db[i])):
            cur_ai1 = db[i][j][0]
            cur_ai2 = db[i][j][1]
            if h_top_down[i+1].get(cur_ai2) is None:
                h_bottom_up[i][cur_ai2].remove(cur_ai1)
                if len(h_bottom_up[i][cur_ai2]) == 0:
                    del h_bottom_up[i][cur_ai2]

    # Top down pass is not necessary because when we obtain the result by going top down,
    # the dangling tuples would not be included anyway. The top down pass is automatically
    # handled in get_result function.
    # for i in range(1, len(db)):
    #     for j in range(len(db[i])):
    #         cur_ai1 = db[i][j][0]
    #         cur_ai2 = db[i][j][1]
    #         if h_bottom_up[i-1].get(cur_ai1) is None:
    #             h_top_down[i][cur_ai1].remove(cur_ai2)
    #             if len(h_top_down[i][cur_ai1]) == 0:
    #                 del h_top_down[i][cur_ai1]
                
    return h_bottom_up, h_top_down

def get_result(h_top_down):
    """
    Docstring for get_result
    
    :param h_top_down: a hashmap from remove_dangling_tuple function. This is used by the DFS to get the result of the query.
    """
    result = []
    def dfs(i, current_tuple, val):
        # If we have reached the end, append the current tuple to result
        if i == len(h_top_down):
            for j in current_tuple:
                result.append(j + val)
            return
        
        # If we are at the first relation, we need to initialize the current_tuple
        # and call dfs for the next relation
        if i == 0:
            for key in h_top_down[i]:
                for tup in h_top_down[i][key]:
                    dfs(i+1, [[key]], [tup])
                    
        # Else, append to existing values to current_tuple and call dfs for the next relation
        else:
            for key in val:
                if key in h_top_down[i]:
                    for tup in h_top_down[i][key]:
                        dfs(i+1, [i + val for i in current_tuple], [tup])                    
    
    dfs(0, [], [])
    return result

In [3]:
# Test case
R1 = [[1, 10], [2, 20], [3, 30]]
R2 = [[10, 100], [20, 200], [99, 999], [30, 200]]   # (99,999) is dangling
R3 = [[100, 1000], [200, 2000]]

relations = [R1, R2, R3]

h1, h2 = remove_dangling_tuple(relations)
result = get_result(h2)
for r in result:
    print(r)

[1, 10, 100, 1000]
[2, 20, 200, 2000]
[3, 30, 200, 2000]


# Problem 3: Left to right implementation
We implement two versions of this algorithm. The first utilizes hashmaps to speed up key looking up process. The second perform nested loops to perform the join.

In [4]:
# Problem 3 implementation hashmap: This implementation is directly join the line query from left to right using hashmap.
def construct_hashmap_problem3(arr, index):
    """
    Docstring for construct_hashmap_problem3
    
    :param arr: a 2D array
    :param index: the index to be used as key.
    :return: a hashmap where the keys are the elements at the given index, and values are lists of remaining elements in the tuple.
    """
    h = {}
    for i in range(len(arr)):
        cur_element = arr[i]
        key = cur_element[index]
        val = cur_element[:index] + cur_element[index+1:]
        if key not in h:
            h[key] = [val]
        else:
            h[key].append(val)
    return h

def inner_join_left_to_right_efficient(db):
    cur_result = db[0] # Initiate the current result as the first table
    for i in range(1, len(db)):
        temp_result = []
        cur_db = db[i] # Current database
        # Skip if current result is empty
        if len(cur_result) == 0:
            continue
        
        # Construct hashmap for current result based on the last attribute (joining attribute)
        cur_joining_attribute_val = construct_hashmap_problem3(cur_result, len(cur_result[0]) - 1)
        
        # Construct hashmap for current db based on the first attribute (joining attribute)
        h = construct_hashmap_problem2(cur_db, 0)
        
        # Perform the join using the two hashmaps
        for j in cur_joining_attribute_val:
            if j in h:
                for val in cur_joining_attribute_val[j]:
                    for tup in h[j]:
                        temp_result.append(val + [j, tup])
                        
        # Set current result to temp result
        cur_result = temp_result
    return cur_result

In [5]:
# Problem 3 implementation brute force: This implementation is directly join the line query from left to right by nested loops.
def inner_join_left_to_right_brute_force(db):
    cur_result = db[0]
    for i in range(1, len(db)):
        temp_result = []
        cur_db = db[i]
        for j in cur_result:
            for k in cur_db:
                if k[0] == j[-1]:
                    temp_result.append(j + [k[1]])
        cur_result = temp_result
    return cur_result

In [6]:
# Test case for hashmap implementation
result = inner_join_left_to_right_efficient(relations)
for r in result:
    print(r)

[1, 10, 100, 1000]
[2, 20, 200, 2000]
[3, 30, 200, 2000]


In [7]:
# Test case for brute force implementation
result = inner_join_left_to_right_brute_force(relations)
for r in result:
    print(r)

[1, 10, 100, 1000]
[2, 20, 200, 2000]
[3, 30, 200, 2000]


# Problem 4: Build test database to compare run time.

In [8]:
# Problem 4:
R1 = []
for i in range(1, 101):
    R1.append([i, random.randint(1, 5000)])

R2 = []
for i in range(1, 101):
    R2.append([random.randint(1, 5000), i])
    
R3 = []
for i in range(1, 101):
    R3.append([i, i])
    
db = [R1, R2, R3]

In [9]:
start_time = time.perf_counter()
h1, h2 = remove_dangling_tuple(db)
result = get_result(h2)
print("Results from Problem 2's implementation:")
if not result:
    print("No result found.")
else:
    for r in result:
        print(r)
end_time = time.perf_counter()
print(f"Execution time for problem 2's implementation: {end_time - start_time} seconds")

Results from Problem 2's implementation:
[58, 4817, 44, 44]
Execution time for problem 2's implementation: 0.0007525999681092799 seconds


In [10]:
start_time = time.perf_counter()
result = inner_join_left_to_right_efficient(db)
print("Results from Problem 3's implementation using hashmap:")
if not result:
    print("No result found.")
else:
    for r in result:
        print(r)
end_time = time.perf_counter()
print(f"Execution time for problem 3's implementation using hashmap: {end_time - start_time} seconds")

Results from Problem 3's implementation using hashmap:
[58, 4817, 44, 44]
Execution time for problem 3's implementation using hashmap: 0.0003157000173814595 seconds


In [11]:
start_time = time.perf_counter()
result = inner_join_left_to_right_brute_force(db)
print("Results from Problem 3's implementation using bruteforce:")
if not result:
    print("No result found.")
else:
    for r in result:
        print(r)
end_time = time.perf_counter()
print(f"Execution time for problem 3's implementation using bruteforce: {end_time - start_time} seconds")

Results from Problem 3's implementation using bruteforce:
[58, 4817, 44, 44]
Execution time for problem 3's implementation using bruteforce: 0.001156899961642921 seconds


# Problem 5: build test database to compare run time.

In [12]:
# Problem 5 implementation:
R1 = []

for i in range(1, 1001):
    R1.append([i, 5])
for i in range(1001, 2001):
    R1.append([i, 7])
R1.append([2001, 2002])
random.shuffle(R1)


R2 = []
for i in range(1, 1001):
    R1.append([5, i])
for i in range(1001, 2001):
    R1.append([7, i])
R2.append([2002, 8])
random.shuffle(R2)

R3 = []
for i in range(2000):
    R3.append([random.randint(2002, 3000), random.randint(1, 3000)])
R3.append([8, 30])
random.shuffle(R3)

db = [R1, R2, R3]

In [13]:
start_time = time.perf_counter()
h1, h2 = remove_dangling_tuple(db)
result = get_result(h2)
print("Results from Problem 2's implementation:")
if not result:
    print("No result found.")
else:
    for r in result:
        print(r)
end_time = time.perf_counter()
print(f"Execution time for problem 2's implementation: {end_time - start_time} seconds")

Results from Problem 2's implementation:
[2001, 2002, 8, 30]
Execution time for problem 2's implementation: 0.007797300000675023 seconds


In [14]:
start_time = time.perf_counter()
result = inner_join_left_to_right_efficient(db)
print("Results from Problem 3's implementation using hashmap:")
if not result:
    print("No result found.")
else:
    for r in result:
        print(r)
end_time = time.perf_counter()
print(f"Execution time for problem 3's implementation using hashmap: {end_time - start_time} seconds")

Results from Problem 3's implementation using hashmap:
[2001, 2002, 8, 30]
Execution time for problem 3's implementation using hashmap: 0.004151000001002103 seconds


In [15]:
start_time = time.perf_counter()
result = inner_join_left_to_right_brute_force(db)
print("Results from Problem 3's implementation using bruteforce:")
if not result:
    print("No result found.")
else:
    for r in result:
        print(r)
end_time = time.perf_counter()
print(f"Execution time for problem 3's implementation using bruteforce: {end_time - start_time} seconds")

Results from Problem 3's implementation using bruteforce:
[2001, 2002, 8, 30]
Execution time for problem 3's implementation using bruteforce: 0.0012255000183358788 seconds


# Problem 6: Compare with running time in MySQL

In [16]:
mydb = mysql.connector.connect(
      host="localhost",
      user="root",
      password=os.getenv("PASSWORDSQL"),
      database="cs580final"
    )

In [17]:
mycursor = mydb.cursor()

In [18]:
# Create tables. Commented out to avoid re-creation errors.
# mycursor.execute("CREATE TABLE R1 (a1 INT NOT NULL, a2 INT NOT NULL);")
# mycursor.execute("CREATE TABLE R2 (a2 INT NOT NULL, a3 INT NOT NULL);")
# mycursor.execute("CREATE TABLE R3 (a3 INT NOT NULL, a4 INT NOT NULL);")

In [19]:
# Insert data into R1. Commented out to avoid duplicate entries.
empty_query = "TRUNCATE TABLE R1;"
insert_query = "INSERT INTO R1 (a1, a2) VALUES (%s, %s)"

mycursor.execute(empty_query)
mydb.commit()

mycursor.executemany(insert_query, R1)
mydb.commit()

In [20]:
# Insert data into R2.
empty_query = "TRUNCATE TABLE R2;"
insert_query = "INSERT INTO R2 (a2, a3) VALUES (%s, %s)"

mycursor.execute(empty_query)
mydb.commit()

mycursor.executemany(insert_query, R2)
mydb.commit()

In [21]:
# Insert data into R3.
empty_query = "TRUNCATE TABLE R3;"
insert_query = "INSERT INTO R3 (a3, a4) VALUES (%s, %s)"

mycursor.execute(empty_query)
mydb.commit()

mycursor.executemany(insert_query, R3)
mydb.commit()

In [22]:
start_time = time.perf_counter()
mycursor.execute("SELECT r1.a1, r2.a2, r3.a3, r3.a4 FROM r1 join r2 on r1.a2 = r2.a2 join r3 on r2.a3 = r3.a3;")
end_time = time.perf_counter()
print(f"Execution time for MySQL: {end_time - start_time} seconds")
myresult = mycursor.fetchall()
print("Results from MySQL:")
for r in myresult:
    print(r)

Execution time for MySQL: 0.004656300006899983 seconds
Results from MySQL:
(2001, 2002, 8, 30)


In [23]:
mycursor.execute("EXPLAIN SELECT r1.a1, r2.a2, r3.a3, r3.a4 FROM r1 join r2 on r1.a2 = r2.a2 join r3 on r2.a3 = r3.a3;")
myresult = mycursor.fetchall()
print("Query Execution Plan from MySQL:")
for r in myresult:
    print(r)

Query Execution Plan from MySQL:
(1, 'SIMPLE', 'r2', None, 'ALL', None, None, None, None, 1, 100.0, None)
(1, 'SIMPLE', 'r3', None, 'ALL', None, None, None, None, 2001, 10.0, 'Using where; Using join buffer (hash join)')
(1, 'SIMPLE', 'r1', None, 'ALL', None, None, None, None, 4001, 10.0, 'Using where; Using join buffer (hash join)')
